# Phase 1: Scikit-learn

Tên và MSSV của từng thành viên:
- Đinh Viết Lợi - 22120188.
- Nguyễn Trần Lợi - 22120190.
- Nguyễn Nhật Long - 22120194.


---

## Nắm yêu cầu của Phase 1:
Đây là bài toán phân loại nhận biết đoạn âm thanh có chứa tiếng gà tây hay không bằng cách sử dụng thư viện scikit-learn và tuân theo quy trình: Phân tích Dữ liệu Khám phá (Exploratory Data Analysis), Phát triển Mô hình (Model Development) và Đánh giá Mô hình (Model Evaluation).

## Import các thư viện cần thiết

In [69]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve,precision_recall_curve,precision_score,recall_score,f1_score
from sklearn.impute import SimpleImputer


from imblearn.over_sampling import ADASYN
from imblearn.combine import SMOTETomek

## Đọc dữ liệu

In [70]:
path_to_data = "."

In [71]:
with open(path_to_data + "/train.json") as f:
    train_data = json.load(f)

with open(path_to_data + "/test.json") as f:
    test_data = json.load(f)

# Chuyển  dữ liệu thành DataFrame để dễ quan sát
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)


In [72]:
# Dữ liệu huấn luỵện
train_df.head()

,audio_embedding,is_turkey,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[172, 34, 216, 110, 208, 46, 95, 66, 161, 125...",0,kDCk3hLIVXo,70,60
1,"[[169, 20, 165, 102, 205, 62, 110, 103, 211, 1...",1,DPcGzqHoo7Y,40,30
2,"[[148, 8, 138, 60, 237, 48, 121, 108, 145, 177...",1,7yM63MTHh5k,240,230
3,"[[151, 0, 162, 88, 171, 71, 47, 90, 179, 190, ...",1,luG3RmUAxxM,520,510
4,"[[162, 17, 187, 111, 211, 105, 92, 67, 203, 15...",0,PIm3cjxTpOk,10,0


In [73]:
# Dữ liệu test
test_df.head()

,audio_embedding,vid_id,end_time_seconds_youtube_clip,start_time_seconds_youtube_clip
0,"[[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...",pyKh38FXD3E,10,0
1,"[[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...",THhP1idrWXA,40,30
2,"[[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...",jsw3T6GY2Nw,40,30
3,"[[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...",nFkXTMHcjMU,24,14
4,"[[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...",Au8g9kAlrLQ,40,30


## Khám phá dữ liệu

### Các cột có trong dữ liệu

In [74]:
train_df.columns

Index(['audio_embedding', 'is_turkey', 'vid_id',
       'end_time_seconds_youtube_clip', 'start_time_seconds_youtube_clip'],
      dtype='object')

- Dữ liệu trong `train_df` chứa các cột trên. Trong đó, các cột cần thiết để thực hiện huấn luyện là `audio_embedding`, `is_turkey`.

In [75]:
test_df.columns

Index(['audio_embedding', 'vid_id', 'end_time_seconds_youtube_clip',
       'start_time_seconds_youtube_clip'],
      dtype='object')

- `test_df` có các cột trên. Dựa vào yêu cầu, các cột sử dụng là `vid_id`, `audio_embedding`.

## Phát triển mô hình

### Xử lý dữ liệu trước khi huấn luyện

- Vì mỗi `audio_embedding` có số lượng frame khác nhau do đó để dễ dàng trong việc huấn luyện ta thực hiện lấy trung bình của mỗi cột trong các `audio_embedding`.

In [76]:
def combined_embeddings(embeddings): # Hàm dùng để tính toán trên embeddings
    X= np.array(embeddings)
    return np.mean(X, axis=0)


In [77]:
train_X = np.stack(train_df['audio_embedding'].apply(combined_embeddings)) # Lấy trung bình của mỗi cột trong các audio_embedding
train_Y = train_df['is_turkey'].values # Lấy nhãn của dữ liệu huấn luyện

valid_idx = test_df['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_X = np.stack(test_df['audio_embedding'].apply(combined_embeddings)) # Lấy trung bình của mỗi cột trong các audio_embedding

In [78]:
# In ra kích thước của dữ liệu sau khi xử lý
print("Train X shape:", train_X.shape)

# In ra hai dòng đầu tiên của dữ liệu huấn luyện
print(train_X[:2])


Train X shape: (1195, 128)
[[166.6  29.3 198.4 106.4 215.2  57.5  77.3  65.  188.5 149.7  77.3  71.2
  120.  156.1  70.7 105.8 101.1 131.5 175.6  90.7  25.  213.2 106.5 135.5
   47.8 110.3 190.6 183.  104.5  35.6  99.6 109.2 160.  101.   99.5 141.8
  134.9  95.6  90.7  98.6  70.3 150.2  15.  160.5 196.4 140.2 113.9  62.7
   91.2 253.3 184.5  63.5  73.2 155.   30.2  50.1 147.1  38.1 119.5 197.7
  182.6  74.5  59.7 182.9 213.4 187.9 176.7 115.9 251.3 121.2 119.   86.3
  194.4 200.2  50.8 101.1 117.4 109.6 127.5 193.2 148.7 211.7 145.9 160.7
  210.   95.  113.3 232.8  44.2   0.  108.9 221.4 209.5 162.5  91.8 252.5
   98.2   6.1 205.  174.8 126.7  14.3 159.2 119.7  87.7 224.9 120.8 100.
   96.3  99.5 212.3  61.7 102.4 147.9  94.8 190.5 144.5 205.8   1.9  85.2
   44.3 186.2   0.   48.2 238.6 138.5 101.  255. ]
 [173.3  13.7 164.7  93.5 201.3  94.2 122.  114.  166.  181.8  89.5 145.3
   77.  160.1  65.3 105.1 128.4 113.5 121.2 131.3  24.3 201.  104.5  84.5
   93.8 120.4 171.9 179.8 132.3  96

In [89]:
# In ra kích thước của dữ liệu sau khi xử lý
print("Train X shape:", test_X.shape)

# In ra hai dòng đầu tiên của dữ liệu huấn luyện
print(test_X[:2])

Train X shape: (1196, 128)
[[169.7  22.2 220.3 130.3 216.3  85.9 116.3  80.6 127.9 188.8  83.   85.
  106.   79.  104.5  99.3  98.1 152.  201.5  75.   55.5 252.4  60.5  97.1
    4.5 112.1  39.9 168.9  84.8  75.2 157.8  95.2  75.1 115.7 212.1 184.5
   99.6  74.3  56.3 195.4  59.1 125.9  83.  229.  220.1  87.4  85.4  99.8
  106.8 233.6 199.1  16.2 167.8 120.9 111.  108.9  33.8  44.  118.4 186.2
  145.9 167.   92.4 130.  252.6 188.3 205.8 138.  138.5  24.  170.2  84.1
  223.1 177.1  95.5  39.6 168.9 230.7  32.1 229.5  31.4 180.1 232.8 191.8
  220.4  55.7 166.4 239.  133.9 106.7  80.4 210.6 182.2 223.7  89.2 255.
    0.   38.1 202.6 167.1  77.7 152.7 181.6 140.9 145.3 143.5  80.7 195.2
  172.9 134.4  66.8 122.1 106.9 155.5 157.3 182.1  92.2 248.8   5.8 216.3
   35.7 196.4 153.    8.3 249.8 135.5  60.5 255. ]
 [165.3  16.1 192.5 140.7 200.4  85.   65.1  49.4 161.5 182.1  70.6  70.4
  129.9 120.7  62.  108.9 100.2 114.7 224.8 108.1  60.4 239.4 101.  105.4
   21.9 129.1  96.3 248.5 122.6  63.

$\rightarrow$ Dữ liệu sau khi xử lý có kích thước là (1195,128) dối với dữ liệu huấn luyện và (1196,128) đối với dữ liệu test.

- Chuẩn hóa dữ liệu để giúp cho mô hình học dữ liệu tốt hơn.

In [79]:
scaler= StandardScaler()

# Chuyển dữ liệu thành dạng chuẩn
Z = scaler.fit_transform(train_X)
test_Z = scaler.transform(test_X)

In [ ]:
# In ra hai dòng đầu tiên của dữ liệu huấn luyện đã chuẩn hoá
print(Z[:2])


[[ 6.49126134e-01 -3.37372579e-01  6.07047657e-01 -1.55904665e-03
   5.05158521e-01 -1.03328125e+00 -1.03239333e+00 -5.33529786e-01
   3.86531989e-01 -4.47395994e-01 -2.74144379e-02 -4.20736723e-01
  -6.97223581e-01  4.89847279e-02 -1.00862245e-02  5.17695051e-01
  -7.88857255e-01  2.36558405e-03  1.28815688e-01 -1.04025799e+00
  -1.11466184e+00  5.01591717e-01  4.15651542e-02  8.60688786e-01
  -6.90212362e-01 -6.49439468e-01  1.11483499e+00  2.39041901e-01
  -6.60769275e-01 -1.19587928e+00 -4.39144984e-01  7.08349089e-01
   1.41517854e+00 -9.93197161e-02 -4.66426250e-01 -6.10170469e-01
   1.61667615e-01 -6.10943722e-02 -7.29774492e-01 -2.77667246e-01
  -1.82020784e-01  2.16234437e-01 -8.79888737e-01  1.49130003e-01
   1.26327012e+00  2.52060582e-01  2.88889424e-01 -1.39291185e+00
   1.54793181e-01  1.19492337e+00  4.52748485e-01 -5.74569614e-01
  -5.29592477e-01  6.95780735e-01 -1.83243910e+00 -1.14632729e+00
   1.98989108e-01 -4.65682654e-01  5.81285692e-02  2.82555017e-01
   8.47565

In [90]:
# In ra hai dòng dữ liệu test sau khi chuẩn hóa
print(test_Z[:2])

[[ 0.72696882 -0.48391334  1.32520899  0.58756669  0.5320257  -0.14079245
   0.09594964  0.01725138 -1.17851656  0.52010312  0.1545031  -0.09457696
  -1.07529831 -1.56380591  0.79380516  0.32255852 -0.88178299  0.58527484
   0.74139656 -1.38918253 -0.51010108  1.20863284 -1.06504654 -0.23248303
  -1.48778668 -0.59905757 -1.77062087 -0.08376021 -1.14219017 -0.25724183
   0.84874581  0.34529016 -1.0184809   0.2470245   1.5619845   0.47861432
  -0.57206661 -0.59965382 -1.42088658  1.92656512 -0.44676719 -0.24715374
   0.29759524  1.77506666  1.81191393 -1.09434919 -0.43343886 -0.59761433
   0.51018347  0.73290838  0.82282115 -1.55457711  1.43446552 -0.12812805
   0.06898761  0.11347618 -2.16765709 -0.36520611  0.03912919  0.05030953
   0.11748537  1.291639    0.2881942  -0.49702111  1.32390857  0.81273016
   1.26009863  0.56392974 -0.77304311 -1.95128969  0.59407038 -0.84159848
   0.63412812 -0.17095761 -0.47519989 -1.11636111  0.70407161  1.56068124
  -1.51961811  0.97985897 -1.09361792 

- Chia dữ liệu huấn luyện

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập xác thực với tỉ lệ tập xác thực là 70%
train_Z, val_Z, train_Y, val_Y = train_test_split(Z, train_Y, test_size=0.7,random_state=45)

### Lựa chọn mô hình với các tham số hiệu quả nhất

In [82]:
 # SVM với tham số tốt nhất từ GridSearch
model = SVC(
            probability=True,   
            random_state=42
        )

### Huấn luyện 

In [83]:
# Huấn luyện mô hình
model.fit(train_Z, train_Y)

SVC(probability=True, random_state=42)

## Đánh giá mô hình

### Dự đoán trên tập val

In [84]:
# Dự đoán trên validation
y_pred_prob = model.predict_proba(val_Z)[:, 1]
y_pred = model.predict(val_Z)

### Đánh giá

In [85]:
# In các chỉ số đánh giá
print("AUC Score   :", roc_auc_score(val_Y, y_pred_prob))
print("Accuracy    :", accuracy_score(val_Y, y_pred))
print("Precision   :", precision_score(val_Y, y_pred))
print("Recall      :", recall_score(val_Y, y_pred))
print("F1 Score    :", f1_score(val_Y, y_pred))

AUC Score   : 0.9904747703686031
Accuracy    : 0.953405017921147
Precision   : 0.9507692307692308
Recall      : 0.9307228915662651
F1 Score    : 0.9406392694063926


## Dự đoán và lưu lại kết quả trên tập Test

### Dự đoán trên tập test

In [86]:
# Dự đoán trên tập test 
test_pred_prob = model.predict_proba(test_Z)[:, 1]
test_df['is_turkey'] = -1.0
test_df.loc[valid_idx, 'is_turkey'] = test_pred_prob  

In [91]:
# In ra vài dòng dữ liệu sau khi dự đoán
print(test_df.head())

                                     audio_embedding       vid_id  \
0  [[177, 20, 226, 132, 198, 81, 111, 59, 132, 18...  pyKh38FXD3E   
1  [[169, 21, 204, 161, 195, 72, 60, 39, 152, 184...  THhP1idrWXA   
2  [[165, 13, 198, 141, 199, 81, 173, 54, 119, 11...  jsw3T6GY2Nw   
3  [[167, 18, 188, 159, 198, 63, 156, 36, 179, 22...  nFkXTMHcjMU   
4  [[178, 32, 181, 100, 198, 46, 82, 83, 136, 227...  Au8g9kAlrLQ   

   end_time_seconds_youtube_clip  start_time_seconds_youtube_clip  is_turkey  
0                             10                                0   0.019094  
1                             40                               30   0.010089  
2                             40                               30   0.086529  
3                             24                               14   0.005280  
4                             40                               30   0.769496  


### Lưu lại kết quả

In [87]:
# Xuất file kết quả
test_df.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('svm_result.csv', index=False)